## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Pisco,-13.7000,-76.2167,61.02,94,99,1.14,PE
1,1,Zhirnov,48.1709,41.1261,66.13,42,100,8.86,RU
2,2,Hanmer Springs,-42.5167,172.8167,46.49,94,81,1.52,NZ
3,3,Ushuaia,-54.8000,-68.3000,47.86,66,75,10.36,AR
4,4,Pangoa,-12.1167,-73.0000,68.13,97,100,1.23,PE


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))


What is your desired minimum temperature for your trip? 69
What is your desired maximum temperature for your trip? 100


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
6,6,Puerto Ayora,-0.7393,-90.3518,74.59,94,94,4.88,EC
7,7,Metro,-5.1131,105.3067,78.22,82,57,6.29,ID
9,9,Atuona,-9.8000,-139.0333,78.89,78,41,16.46,PF
11,11,Hermanus,-34.4187,19.2345,74.80,48,7,14.00,ZA
12,12,Bethanien,-26.5020,17.1583,81.68,29,0,6.13,NaN
15,15,Komsomolskiy,40.4272,71.7189,84.92,13,81,4.61,UZ
16,16,Intipuca,13.1969,-88.0544,77.86,78,65,0.94,SV
18,18,Butaritari,3.0707,172.7902,81.75,75,57,12.55,KI
24,24,Rikitea,-23.1203,-134.9692,77.90,70,6,12.41,PF
25,25,Bredasdorp,-34.5322,20.0403,70.61,41,2,12.35,ZA


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       3
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
6,6,Puerto Ayora,-0.7393,-90.3518,74.59,94,94,4.88,EC
7,7,Metro,-5.1131,105.3067,78.22,82,57,6.29,ID
9,9,Atuona,-9.8000,-139.0333,78.89,78,41,16.46,PF
11,11,Hermanus,-34.4187,19.2345,74.80,48,7,14.00,ZA
15,15,Komsomolskiy,40.4272,71.7189,84.92,13,81,4.61,UZ
...,...,...,...,...,...,...,...,...,...
574,574,Grand Gaube,-20.0064,57.6608,80.38,88,40,8.01,MU
577,577,Inongo,-1.9500,18.2667,80.04,73,99,3.27,CD
578,578,Bitung,1.4451,125.1824,79.65,82,99,5.70,ID
579,579,Yeppoon,-23.1333,150.7333,71.38,72,0,13.04,AU


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,74.59,-0.7393,-90.3518,
7,Metro,ID,78.22,-5.1131,105.3067,
9,Atuona,PF,78.89,-9.8000,-139.0333,
11,Hermanus,ZA,74.80,-34.4187,19.2345,
15,Komsomolskiy,UZ,84.92,40.4272,71.7189,
16,Intipuca,SV,77.86,13.1969,-88.0544,
18,Butaritari,KI,81.75,3.0707,172.7902,
24,Rikitea,PF,77.90,-23.1203,-134.9692,
25,Bredasdorp,ZA,70.61,-34.5322,20.0403,
32,Mahebourg,MU,79.25,-20.4081,57.7000,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()


,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,74.59,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Metro,ID,78.22,-5.1131,105.3067,OYO 3177 Wisma Lintang
9,Atuona,PF,78.89,-9.8000,-139.0333,Villa Enata
11,Hermanus,ZA,74.80,-34.4187,19.2345,Misty Waves Boutique Hotel
15,Komsomolskiy,UZ,84.92,40.4272,71.7189,Ikathouse


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt></dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))